# Dictionary stuff

In [1]:
d1 = {'k1':1, 'k2':2, 'k3':3}

d1

{'k1': 1, 'k2': 2, 'k3': 3}

In [2]:
for item in d1:
    print(item)

k1
k2
k3


In [3]:
[item for item in d1]

['k1', 'k2', 'k3']

In [4]:
for item in d1:
    print(d1[item])

1
2
3


In [5]:
[d1[item] for item in d1]

[1, 2, 3]

In [6]:
import pandas as pd

In [7]:
df = pd.read_csv('datasets/Credit_Data_Raw.csv')
df

,CHK_ACCT,DURATION,HISTORY,NEW_CAR,USED_CAR,FURNITURE,RADIO_TV,EDUCATION,RETRAINING,AMOUNT,...,AGE,OTHER_INSTALL,RENT,OWN_RES,NUM_CREDITS,JOB,NUM_DEPENDENTS,TELEPHONE,FOREIGN,DEFAULT
0,0,6,4,0,0,0,1,0,0,1169,...,67,0,0,1,2,2,1,1,0,0
1,1,48,2,0,0,0,1,0,0,5951,...,22,0,0,1,1,2,1,0,0,1
2,3,12,4,0,0,0,0,1,0,2096,...,49,0,0,1,1,1,2,0,0,0
3,0,42,2,0,0,1,0,0,0,7882,...,45,0,0,0,1,2,2,0,0,0
4,0,24,3,1,0,0,0,0,0,4870,...,53,0,0,0,2,2,2,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,3,12,2,0,0,1,0,0,0,1736,...,31,0,0,1,1,1,1,0,0,0
996,0,30,2,0,1,0,0,0,0,3857,...,40,0,0,1,1,3,1,1,0,0
997,3,12,2,0,0,0,1,0,0,804,...,38,0,0,1,1,2,1,0,0,0
998,0,45,2,0,0,0,1,0,0,1845,...,23,0,0,0,1,2,1,1,0,1


In [8]:
x = df.drop('DURATION', axis=1)[['HISTORY', 'NEW_CAR']]
x

,HISTORY,NEW_CAR
0,4,0
1,2,0
2,4,0
3,2,0
4,3,1
...,...,...
995,2,0
996,2,0
997,2,0
998,2,0


# Model Function Testing

In [171]:
'''
Credit Crunch
Author: Andrew McKinney
Creation Date: 2020-04-28
'''




def credit_crunch(converted_data):
# Credit Crunch is a TensorFlow Neural Network that predicts an applicant's probablity to default on a loan. 
# If default is predicted, a loan denial is returned; otherwise, approved.
# The NN model is dynamically created everytime to match in input data that is imported as a key:value dictionary.
# Generic NN model parameters can be set in the DEV TOOLS.


    ### DEV TOOLS ###
    return_model_evaluation = True
    numpy_seed = 42
    number_inputs = len(converted_data)
    number_classes = 2
    number_hidden_layers = 2
    number_hidden_nodes = 100
    number_epochs = 50
    layer_activation = 'relu'
    classifier_activation = 'softmax'
    learn_metrics = ['accuracy']
    loss_type = 'categorical_crossentropy'
    optimizer_type = 'adam'

    # import dependencies
    import numpy as np
    import pandas as pd
    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import LabelEncoder, MinMaxScaler
    from tensorflow.keras.utils import to_categorical
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Dense



    # setting numpy seed for reproducible results
    np.random.seed(numpy_seed)


    # import train data
    raw_data = pd.read_csv('datasets/Credit_Data_Raw.csv')

    raw_data.dropna()
    
    # defining labels, input fields, and input form data
    X = raw_data.drop('DEFAULT', axis=1)[[item for item in converted_data]]
    y = np.array(raw_data['DEFAULT']).reshape(-1, 1)
    data_bundle = np.array(list([converted_data[item] for item in converted_data])).reshape(1, -1)
    
    # spliting data to test and training sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
    
    
    # scaling data 
    X_scaler = MinMaxScaler().fit(X_train)
    X_train_scaled = X_scaler.transform(X_train)
    X_test_scaled = X_scaler.transform(X_test)
    data_bundle_scaled = X_scaler.transform(data_bundle)
    

    # one-hot-encoding output labels
    label_encoder = LabelEncoder()
    label_encoder.fit(y_train)
    encoded_y_train = label_encoder.transform(y_train)
    encoded_y_test = label_encoder.transform(y_test)
    y_train_categorical = to_categorical(encoded_y_train)
    y_test_categorical = to_categorical(encoded_y_test)

    # instantiating Neural Net Model
    model = Sequential()

    # adding input layer
    model.add(Dense(units=number_hidden_nodes, activation=layer_activation, input_dim=number_inputs))

    # adding hidden layers
    for layer in np.arange(0, number_hidden_layers):
        model.add(Dense(units=number_hidden_nodes, activation=layer_activation))

    # adding classifier layer
    model.add(Dense(units=number_classes, activation=classifier_activation))

    # compiling model
    model.compile(optimizer=optimizer_type, loss=loss_type, metrics=learn_metrics)

    # fitting model to training data
    model.fit(
        X_train_scaled,
        y_train_categorical,
        epochs=number_epochs,
        shuffle=True,
        verbose=0
    )


    # predicting approval for user
    crunchies = model.predict(data_bundle)


    # returning model evaluation if turned on
    if return_model_evaluation:
        model_loss, model_accuracy = model.evaluate(X_test_scaled, y_test_categorical, verbose=2)

        return crunchies, model_loss, model_accuracy

    else:

        return crunchies


In [174]:
import numpy as np

t1 = df.drop('DURATION', axis=1).iloc[0]

# t1 = [item for item.column() in t1]

c1 = df.columns

dp = {}

for index, item in enumerate(df.columns.drop(['DEFAULT'])):
    dp[item] = t1[index]

dp

{'CHK_ACCT': 0,
 'DURATION': 4,
 'HISTORY': 0,
 'NEW_CAR': 0,
 'USED_CAR': 0,
 'FURNITURE': 1,
 'RADIO_TV': 0,
 'EDUCATION': 0,
 'RETRAINING': 1169,
 'AMOUNT': 4,
 'SAV_ACCT': 4,
 'EMPLOYMENT': 4,
 'INSTALL_RATE': 0,
 'MALE_DIV': 1,
 'MALE_SINGLE': 0,
 'MALE_MAR_or_WID': 0,
 'CO_APPLICANT': 0,
 'GUARANTOR': 4,
 'PRESENT_RESIDENT': 1,
 'REAL_ESTATE': 0,
 'PROP_UNKN_NONE': 67,
 'AGE': 0,
 'OTHER_INSTALL': 0,
 'RENT': 1,
 'OWN_RES': 2,
 'NUM_CREDITS': 2,
 'JOB': 1,
 'NUM_DEPENDENTS': 1,
 'TELEPHONE': 0,
 'FOREIGN': 0}

In [172]:
print(f'Number of Inputs:{len(dp)}')
test, ml, ma= credit_crunch(dp)

print(test, ml, ma)

Number of Inputs:28


C:\Users\armck\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\armck\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


250/250 - 0s - loss: 2.0329 - accuracy: 0.6960
[[0. 1.]] 2.0328604907989503 0.696


In [175]:
[item for item in dp]

['CHK_ACCT',
 'DURATION',
 'HISTORY',
 'NEW_CAR',
 'USED_CAR',
 'FURNITURE',
 'RADIO_TV',
 'EDUCATION',
 'RETRAINING',
 'AMOUNT',
 'SAV_ACCT',
 'EMPLOYMENT',
 'INSTALL_RATE',
 'MALE_DIV',
 'MALE_SINGLE',
 'MALE_MAR_or_WID',
 'CO_APPLICANT',
 'GUARANTOR',
 'PRESENT_RESIDENT',
 'REAL_ESTATE',
 'PROP_UNKN_NONE',
 'AGE',
 'OTHER_INSTALL',
 'RENT',
 'OWN_RES',
 'NUM_CREDITS',
 'JOB',
 'NUM_DEPENDENTS',
 'TELEPHONE',
 'FOREIGN']